# Episode 7: Data exploration with pandas

**Teaching:** 30 min   
**Practice:** 20 min   
**Questions:**
- How do I explore heterogeneous data?
- how do I get statistical information out of my data?

**Objectives:**
- Describe what pandas is
- Use pandas to get basic statistical information
- Find correlations between variables
- Clean the data in preparation for further analys

**Key points:**
- Pandas is the fundamental tool in Python for data science: machine learning and deep learning.
- You can use it for much simpler data analys

# Pandas

- [Pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/index.html#intro-to-pandas) was created to leverage the data exploration and statistical capabilities of R
- Indeed, Pandas documentation includes [detailed cheatsheets](https://pandas.pydata.org/pandas-docs/stable/getting_started/index.html#coming-from) to help you with the transition from R, among others. 
- While still a bit behind in terms of speciallized libraries in the field of bioinformatics, pandas has become the dominant, base tool for machine learning and deep learning.
- Here we will explore a very minimal part of all of pandas capabilities. Have a look at the [User Guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html) for a complete description of pandas. 
- Pandas is imported with

```python
import pandas as pd
```

# DataFrame and Series

- There are two basic building blocks

### Series:

- Is a one dimensional structure containing objects of the same type.
- Typically contains the different observations of a single variable (eg: age of a population of subjects, or their sex, or their country or origin.)
- An index identifies each of the observations. The index can be an integer number, but also dates, times or many other python objects.

### Dataframe:

- Is (normally) a two dimensional structure containing one or more Series that act as columns.
- Each of the Series can have data of different type.
- All series must have the same index. If originally, they didn't, `NaN` values are added. 
- Each column has a name that can be used to directly access that Series. 

# Exploring data: The Titanic passengers

- To explore the capabilities of pandas, we will use the (partial) [list of passengers of the Titanic](https://public.opendatasoft.com/explore/dataset/titanic-passengers/table/)
- To start, load the file `titanic-passengers.csv` with

```python
df = pd.read_csv("Data/titanic-passengers.csv", delimiter=";")
```

- Now, let's gather some general information on the data using `head`, `tail` and `info`

## Selecting and filtering data

- Individual columns can be access using their name, either as dictionary keys or attributes.
- Multiple columns can be selected at the same time enclosing their names in double square brackets.
    
```python
df.Age
df["Age"]
df[["Age", "Survived"]]
```
- `loc` can be used to select a specific row or range of rows using the index

```python
df.loc[7]
df.loc[35:37]
```

- `iloc` can be used to slect specific rows and columns, or individual elements, like in numpy

```python
df.iloc[7, 3]
df.iloc[7:10, 3:5]
```

- Logical expresions can be used to do more complex selections

```python
df[df["Fare"] > 100]
df[df["Pclass"].isin([2, 3])]
df[(df["Age"] > 50) & (df["Survived"] == "Yes")]
```

## Getting statistical information

- Pandas provides the usual statistical metrics like mean, standard deviation, etc. [More info](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html#descriptive-statistics). 
- A general summary of aggregated statistics can be obtained using `describe`.

```python
df[["Age", "Fare"]].describe()
```

- Statistical information can also be obtained by grouping the data by category.

```python
df[["Age", "Sex"]].groupby("Sex").mean()
df.groupby("Sex")["Age"].mean()
```

# Modifiying the data

- So far we have only explored the data, but this often needs to be modified or curated before analysis.
- A new series with new data based on that of other columns can be added as if the dataframe were a dictionary

```python
df["Children"] = df["Age"] <= 12
```

- Some Series appear to be numbers(like `Pclass`), or strings (`Sex`, `Survived`), but are actually a finite set of categories. We should change them to categorical data.

```python
df["Pclass"] = df["Pclass"].astype('category')
df["Sex"] = df["Sex"].astype('category')
df["Survived"] = df["Survived"].astype('category')
```

- The "Embarked" column indicates the city where the passenger embarked the Titanic, but it just sais "S", "C" and "Q", which is a bit obscure. Let's give it more human readable names.

```python
df.loc[(df.Embarked == "S"),"Embarked"]="Southampton"
df.loc[(df.Embarked == "C"),"Embarked"]="Cherbourg"
df.loc[(df.Embarked == "Q"),"Embarked"]="Queenstown"
```

- Finally, for further analysis, it is convenient to have the survival information as numbers: 1 if survived and 0 otherwise

```python
df["Survived_num"] = (df["Survived"] == "Yes").astype(int)
```

# Would I survive?

- Now, the whole point of data analysis is to learn new things. To do that properly, we should use proper machine learning packages (eg. [scikit-learn](https://scikit-learn.org/stable/index.html)), but we can get some hints using pandas.
- For that, we use a [pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)
    - Useful to find correlations between columns aggregating values from other column.
    - Any number of columns can be compared to each other.
    - The values to aggregate must be numeric.
    
```python
df.pivot_table(values='Survived_num', index='Sex', columns="Pclass", aggfunc=np.mean)
```

- To what extent are those numbers significative - or if they are relevant at all - requires further analysis, but just a quick check can tell us a lot:

```python
df.groupby("Sex")["Pclass"].value_counts()
```

# Exercise 1

- Use the pivot table to find other correlations for example:
    - With the city passengers embarked, in addition to sex and Pclass.
    - Use as aggregate function the standard deviation (np.std). What can you learn from that?